In [2]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = "PK0S8EZU85JID2Y28MRA"
secret_key = "JdD6SgkPEIimjUIGrwerKcLNGcxFh7Xdx5n9NWvJ"

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [4]:
import alpaca_trade_api as tradeapi

BASE_URL = "https://paper-api.alpaca.markets"

# Initialize API
api = tradeapi.REST(api_key, secret_key, BASE_URL, api_version="v2")

# Close all open positions
try:
    api.close_all_positions()
    print("Closed all positions successfully.")
except Exception as e:
    print(f"Error closing positions: {e}")

# Cancel all open orders
try:
    api.cancel_all_orders()
    print("Canceled all open orders successfully.")
except Exception as e:
    print(f"Error canceling orders: {e}")

# Reset cash balance (Alpaca does not provide an API method for this)
print("To reset cash balance, manually adjust it in the Alpaca dashboard.")

Closed all positions successfully.
Canceled all open orders successfully.
To reset cash balance, manually adjust it in the Alpaca dashboard.


In [6]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [8]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [10]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)

In [12]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [14]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [16]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct_config = trade_client.get_account_configurations()

In [18]:
# set account configuration
# ref. https://docs.alpaca.markets/reference/patchaccountconfig-1
req = acct_config
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_new = trade_client.set_account_configurations(req)

# revert changes
req = acct_config_new
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_reverted = trade_client.set_account_configurations(req)

In [20]:
# get list of assets which are us_equity (default), active, and in NASDAQ
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  # asset_class=AssetClass.US_EQUITY,  # default asset_class is us_equity
  status=AssetStatus.ACTIVE,
  exchange=AssetExchange.NASDAQ,
)
assets = trade_client.get_all_assets(req)

# Trading Strategy

In [23]:
import os
import pickle
import time
import schedule
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import numpy as np


# Stocks for Trading & Tracking： we have different strategies on etfs and stocks
stocks = {
    "NVDA": "/Users/artemislu/Desktop/lstm_local_extrema_NVDA.keras",
    "C":    "/Users/artemislu/Desktop/lstm_local_extrema_C.keras",
    "SBUX": "/Users/artemislu/Desktop/lstm_local_extrema_SBUX.keras"
}
etfs = ["SPY", "VIS"]
symbols = list(stocks.keys()) + etfs
print(symbols )

['NVDA', 'C', 'SBUX', 'SPY', 'VIS']


## Fetch the real time data

In [25]:
import logging
import traceback
from requests.exceptions import RequestException

# Fetching and Saving Stock Data
def get_real_time_market_data(symbol):
    now = datetime.now(ZoneInfo("America/New_York"))
    try:
        req = StockBarsRequest(
            symbol_or_symbols=[symbol],
            timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute),
            start=now - timedelta(days=7)
        )
            
        df = stock_historical_data_client.get_stock_bars(req).df
        return df
    except Exception as e:
        print(f"Failed to fetch real-time data for {symbol}: {e}")
        return None
        
from alpaca.data.historical.stock import StockHistoricalDataClient

stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

def fetch_and_save_data():
    now = datetime.now(ZoneInfo("America/New_York"))
    for symbol in symbols:
        try:
            req = StockBarsRequest(
                symbol_or_symbols=[symbol],
                timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute),
                start=now - timedelta(days=7)
            )
            
            df = stock_historical_data_client.get_stock_bars(req).df

            # Define file paths
            pickle_file_path = os.path.join("/Users/artemislu/Desktop", f"{symbol}.pkl")
            csv_file_path = os.path.join("/Users/artemislu/Desktop", f"{symbol}.csv")

            # Save to Pickle
            with open(pickle_file_path, "wb") as f:
                pickle.dump(df, f)

            # Save to CSV
            df.to_csv(csv_file_path, index=True)

            logging.info(f"Updated {symbol}.csv and {symbol}.pkl at {datetime.now()}")

        except Exception as e:
            logging.error(f"Error updating {symbol}: {e}")
            logging.error(traceback.format_exc())



In [27]:
# Place Orders
def place_order(symbol, qty, side):
    if qty <= 0:
        print(f"{symbol} - Order not placed. Quantity is zero or negative.")
        return
    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type="market",
            time_in_force="gtc"
        )
        print(f"{symbol} - Order Placed: {side.upper()} {qty} shares")
    except Exception as e:
        print(f"{symbol} - Order Failed: {e}")

## ETFS Strategy

In [17]:
# ETF Trading Parameters
etfs = ["SPY", "VIS"]
initial_buy_notional = 20000.0
rebuy_notional = 1000.0
threshold_sell_20 = 0.003
threshold_sell_40 = 0.004
threshold_sell_all = 0.005
threshold_rebuy = -0.01

holding_etf = {symbol: 0.0 for symbol in etfs}
last_buy_price = {}
cash_etf = {}

In [18]:
# ETF Trading Logic
def execute_etf_trading():
    global last_buy_price, holding_etf, cash_etf

    now = datetime.now(ZoneInfo("America/New_York"))

    # Ensure ETF trading only happens during market hours (9:30 AM - 4:00 PM ET)
    if now.hour < 9 or now.hour >= 16:
        return  

    for symbol in etfs:
        df = get_real_time_market_data(symbol)
        if df is None or df.empty:
            continue

        # Get current price information
        market_open_price = df["open"].iloc[-1] if now.hour == 9 and now.minute == 30 else None
        current_price = df["close"].iloc[-1]

        # Initialize first purchase logic at market open
        if symbol not in last_buy_price:
            first_buy_price = market_open_price if market_open_price is not None else current_price
            last_buy_price[symbol] = first_buy_price  # Store first buy price
            holding_etf[symbol] = int(initial_buy_notional / first_buy_price)  # Ensure integer shares
            cash_etf[symbol] = -initial_buy_notional

            # Execute market order to buy ETFs
            if holding_etf[symbol] > 0:
                place_order(symbol, qty=holding_etf[symbol], side="buy")
                print(f"{symbol} First Buy: ${initial_buy_notional}, Price: {first_buy_price:.2f}")

        # Calculate price change from last purchase price
        price_change = (current_price / last_buy_price[symbol]) - 1

        # Selling Logic
        if holding_etf[symbol] > 0:
            if price_change >= threshold_sell_all:
                # Sell all holdings
                place_order(symbol, qty=holding_etf[symbol], side="sell")
                cash_etf[symbol] += holding_etf[symbol] * current_price
                print(f"Sold ALL {symbol} at {current_price:.2f}, Total Cash: {cash_etf[symbol]:.2f}")
                holding_etf[symbol] = 0
                
            elif price_change >= threshold_sell_40:
                # Sell 40% of holdings
                sell_qty = int(holding_etf[symbol] * 0.4)
                if sell_qty > 0:
                    place_order(symbol, qty=sell_qty, side="sell")
                    cash_etf[symbol] += sell_qty * current_price
                    holding_etf[symbol] -= sell_qty
                    print(f"Sold 40% {symbol} at {current_price:.2f}, Remaining: {holding_etf[symbol]} shares")
                    
            elif price_change >= threshold_sell_20:
                # Sell 20% of holdings
                sell_qty = int(holding_etf[symbol] * 0.2)
                if sell_qty > 0:
                    place_order(symbol, qty=sell_qty, side="sell")
                    cash_etf[symbol] += sell_qty * current_price
                    holding_etf[symbol] -= sell_qty
                    print(f"Sold 20% {symbol} at {current_price:.2f}, Remaining: {holding_etf[symbol]} shares")

        # Rebuy Logic (if price drops 1% below the last purchase price)
        if holding_etf[symbol] == 0 and price_change <= threshold_rebuy:
            rebuy_shares = int(rebuy_notional / current_price)
            if rebuy_shares > 0:
                place_order(symbol, qty=rebuy_shares, side="buy")
                holding_etf[symbol] = rebuy_shares
                cash_etf[symbol] -= rebuy_notional
                last_buy_price[symbol] = current_price  # Update last purchase price
                print(f"Rebought {symbol}: {rebuy_shares} shares at {current_price:.2f}, Total Cash: {cash_etf[symbol]:.2f}")




## Stock Strategy

# Strategy Execution

In [21]:
print("Current keys in scalers dictionary:", scalers.keys())


NameError: name 'scalers' is not defined

In [ ]:
print(scalers)

In [ ]:
print(scalers['SBUX'])

In [ ]:
print(symbol)

In [29]:
import pandas as pd
import numpy as np
import scipy.signal
import time
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import joblib


# Initialize portfolio for stock "C"
portfolio_cash = 10000  # Starting capital
holdings = 0
active_trade = None  # To track open trades
trade_times_dict = {"buy_times": [], "sell_times": [], "half_sell_times": []}  # ✅ Fix here
symbol = "C"

# Load the MinMaxScaler for "C"
scaler = joblib.load("/Users/artemislu/Desktop/scaler_c.pkl")
model = tf.keras.models.load_model("/Users/artemislu/Desktop/lstm_local_extrema_C.keras")

# Compute technical indicators
def calculate_technical_indicators(df):
    df["SMA_6H"] = df["close"].rolling(window=360, min_periods=1).mean()
    df["EWMA_6H"] = df["close"].ewm(span=360, adjust=False).mean()
    df["momentum_2H"] = df["close"].diff(periods=120)

    delta = df["close"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=360, min_periods=1).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=360, min_periods=1).mean()
    rs = gain / (loss + 1e-9)
    df["RSI_6H"] = 100 - (100 / (1 + rs))

    df.fillna(method="ffill", inplace=True)
    df.fillna(method="bfill", inplace=True)
    return df

# Prepare and scale data
def prepare_and_scale(df, scaler, features):
    df = calculate_technical_indicators(df)
    df.dropna(inplace=True)
    df[features] = scaler.fit_transform(df[features])
    return df

# Predict next 30 minutes prices
def predict_next_30_minutes(df, model, scaler, features):
    if len(df) < 120:
        return None
    data_3d = df[features].iloc[-120:].values.reshape((1, 120, len(features)))
    y_pred_scaled = model.predict(data_3d)
    
    y_pred_actual = []
    for i in range(30):
        dummy_row = np.zeros((1, len(features)))
        dummy_row[0, 0] = y_pred_scaled[0, i]
        inv = scaler.inverse_transform(dummy_row)
        y_pred_actual.append(inv[0, 0])

    return np.array(y_pred_actual)

# Identify buy/sell signals
def find_trade_times(predicted_prices):
    """
    Identifies the absolute minimum (buy) and maximum (sell) prices in the 30-minute prediction window.
    """
    if predicted_prices is None or len(predicted_prices) == 0:
        return [], []

    # Find the index of the absolute minimum price (Best Buy Time)
    min_index = np.argmin(predicted_prices)

    # Find the index of the absolute maximum price (Best Sell Time)
    max_index = np.argmax(predicted_prices)

    return [min_index], [max_index]  # Convert to list format for consistency


# Predict and store predictions every 30 minutes
def predict_and_store():
    """
    Predicts stock prices for the next 30 minutes, stores them, and prints buy/sell signals immediately.
    """
    global latest_predictions, buy_times, sell_times, trade_times_dict
    now = datetime.now()

    # Fetch real-time market data
    df = get_real_time_market_data(symbol)

    # Scale the data
    df_scaled = prepare_and_scale(df, scaler, ["close", "SMA_6H", "EWMA_6H", "momentum_2H", "RSI_6H", "vwap"])

    # Make predictions
    latest_predictions = predict_next_30_minutes(df_scaled, model, scaler, ["close", "SMA_6H", "EWMA_6H", "momentum_2H", "RSI_6H", "vwap"])

    if latest_predictions is not None:
        print(f"\n📈 Updated Prediction at {now.strftime('%H:%M')}:")
        print(latest_predictions)

        # Find absolute buy and sell times (min and max price index)
        buy_times, sell_times = find_trade_times(latest_predictions)

        # Ensure we have valid buy/sell signals before proceeding
        if buy_times and sell_times:
            # Convert index positions to actual timestamps
            buy_time_actual = now + timedelta(minutes=int(buy_times[0]))
            sell_time_actual = now + timedelta(minutes=int(sell_times[0]))

            # Calculate half-sell time (midpoint between buy and sell)
            half_sell_time_actual = buy_time_actual + (sell_time_actual - buy_time_actual) / 2

            trade_times_dict["buy_times"] = [buy_time_actual.strftime('%H:%M')]
            trade_times_dict["sell_times"] = [sell_time_actual.strftime('%H:%M')]
            trade_times_dict["half_sell_times"] = [half_sell_time_actual.strftime('%H:%M')]

            # Print Buy/Sell/Half-Sell times immediately
            print("\n📌 Trading Signals (Based on Prediction):")
            print(f"🟢 Buy Time: {trade_times_dict['buy_times']}")
            print(f"🔴 Sell Time: {trade_times_dict['sell_times']}")
            print(f"🟠 Half-Sell Time: {trade_times_dict['half_sell_times']}")
        else:
            print("⚠️ No valid buy/sell signals found in predictions.")



def execute_stock_trading():
    """
    Executes stock trading strategy:
    - Executes buy/sell orders based on LSTM predictions.
    - Uses batch trading for both buying and selling.
    - Integrates risk management (stop-loss, take-profit, trailing stop).
    - Ensures predictions are only used for a fixed 30-minute window.
    """
    now = datetime.now(ZoneInfo("America/New_York"))

    # Stop trading one hour before market close (After 3:00 PM ET)
    #if now.hour >= 15:
        #print("Stock trading halted. Last hour before market close.")
        #return  


    for symbol in stocks.keys():
        df = get_real_time_market_data(symbol)

        predicted_prices = latest_predictions.get(symbol)
        
    if not latest_predictions:
        print("No predictions available. Skipping stock trading.")
        return

        # Find local minima and maxima for trading signals
        # Execute Buy Orders with Batch Strategy
        if now in trade_times_dict["buy_times"]:
            if now.strftime('%H:%M') == trade_times[buy_time].strftime('%H:%M'):
                buy_amount = portfolio_cash[symbol] * risk_percentage  # 20% of available cash
                print('buy_amount',buy_amount)
                qty_to_buy = int(buy_amount / predicted_prices[buy_time])  # Calculate shares

                if qty_to_buy > 0:
                    place_order(symbol, qty=qty_to_buy, side="buy")
                    holdings[symbol] += qty_to_buy
                    active_trades[symbol] = {
                    "buy_price": predicted_prices[buy_time],
                    "highest_price": predicted_prices[buy_time],
                    "trade_time": now  # Use the current time
                    }
                    
                    print(f"{symbol}: Bought {qty_to_buy} shares at {predicted_prices[buy_time]:.2f}")

        # Execute Full Sell Orders
        for sell_time in sell_times:
            if now.strftime('%H:%M') == trade_times[sell_time].strftime('%H:%M') and holdings[symbol] > 0:
                place_order(symbol, qty=holdings[symbol], side="sell")
                portfolio_cash[symbol] += holdings[symbol] * predicted_prices[sell_time]  
                holdings[symbol] = 0  
                print(f"{symbol}: Sold ALL at {predicted_prices[sell_time]:.2f}")

        # Execute Partial Sell Orders (Sell 30%)
        for i, half_sell_time in enumerate(half_sell_times):
            if half_sell_time is None or holdings[symbol] == 0:
                continue
            if now.strftime('%H:%M') == trade_times[half_sell_time].strftime('%H:%M'):
                qty_to_sell = int(holdings[symbol] * 0.3)  
                if qty_to_sell > 0:
                    place_order(symbol, qty=qty_to_sell, side="sell")
                    holdings[symbol] -= qty_to_sell  
                    print(f"{symbol}: Sold 30% ({qty_to_sell} shares) at {predicted_prices[half_sell_time]:.2f}")

        # Risk Management Execution (After Buying)
        for symbol, trade_data in active_trades.items():
            current_price = get_real_time_market_data(symbol)["close"].iloc[-1]
            highest_price = max(trade_data["highest_price"], current_price)

            action = manage_risk(
                symbol, 
                trade_data["buy_price"], 
                current_price, 
                highest_price, 
                holdings, 
                trade_data["trade_time"]
            )

            if action == "sell_all":
                place_order(symbol, qty=holdings[symbol], side="sell")
                holdings[symbol] = 0
                print(f"{symbol}: Sold all at {current_price:.2f} due to risk management.")
            
            elif action == "sell_partial":
                qty_to_sell = int(holdings[symbol] * 0.3)  # Sell 30% first
                if qty_to_sell > 0:
                    place_order(symbol, qty=qty_to_sell, side="sell")
                    holdings[symbol] -= qty_to_sell
                    print(f"{symbol}: Sold 30% at {current_price:.2f}, remaining {holdings[symbol]} shares.")





# Simulate placing buy/sell orders
def place_order(qty, side):
    global portfolio_cash, holdings, active_trade
    price = get_real_time_market_data(symbol)["close"].iloc[-1]
    
    if side == "buy":
        cost = qty * price
        if portfolio_cash >= cost:
            portfolio_cash -= cost
            holdings += qty
            active_trade = {"buy_price": price, "trade_time": datetime.now()}
            return f"Bought {qty} shares at {price:.2f}"
        return  "Not enough cash to buy"
    
    elif side == "sell":
        if holdings >= qty:
            portfolio_cash += qty * price
            holdings -= qty
            return f"Sold {qty} shares at {price:.2f}"
        return "Not enough shares to sell"
    
    return "Invalid Order"

# **Run the simulation for 10 minutes**
start_time = time.time()

# **Predict at the beginning**
predict_and_store()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step

📈 Updated Prediction at 23:42:
[66.69581739 66.81379443 66.86883721 66.84928124 66.91484696 67.05812222
 67.03287372 66.91990062 67.1330254  66.97065731 66.99374206 66.93295442
 67.12266483 67.02538326 67.0874911  66.96468039 67.10546546 66.93828212
 67.10187652 67.10988555 67.30735174 67.1093365  67.30841234 67.46028838
 67.25556379 67.22204168 67.12695736 67.39452582 67.40892633 67.37806175]

📌 Trading Signals (Based on Prediction):
🟢 Buy Time: ['23:42']
🔴 Sell Time: ['00:05']
🟠 Half-Sell Time: ['23:54']


In [173]:
start_time = time.time()
while time.time() - start_time < 600:
    execute_stockA_trading()
    time.sleep(60)




📌 DEBUG: Trading Execution Start
Current Trade Signals: {'buy_times': ['23:00', '23:04', '23:06', '23:08', '23:10', '23:12', '23:14', '23:18', '23:23'], 'sell_times': ['22:59', '23:02', '23:05', '23:07', '23:09', '23:11', '23:13', '23:17', '23:20', '23:25'], 'half_sell_times': ['22:59', '23:03', '23:05', '23:07', '23:09', '23:11', '23:13', '23:17', '23:21']}
📌 Current Time (Chicago Time): 2025-03-10 23:05:00-05:00
📌 Current Market Price: 67.01
📌 Checking Buy Time: 2025-03-10 23:00:00-05:00 (Expected Execution Time)
⏳ No match for 2025-03-10 23:00:00-05:00. Current Chicago Time: 2025-03-10 23:05:00-05:00
📌 Checking Buy Time: 2025-03-10 23:04:00-05:00 (Expected Execution Time)
⏳ No match for 2025-03-10 23:04:00-05:00. Current Chicago Time: 2025-03-10 23:05:00-05:00
📌 Checking Buy Time: 2025-03-10 23:06:00-05:00 (Expected Execution Time)
⏳ No match for 2025-03-10 23:06:00-05:00. Current Chicago Time: 2025-03-10 23:05:00-05:00
📌 Checking Buy Time: 2025-03-10 23:08:00-05:00 (Expected Execu

KeyboardInterrupt: 

In [179]:
start_time = time.time()
while time.time() - start_time < 600:
    execute_stockA_trading()
    time.sleep(60)


📌 DEBUG: Trading Execution Start
Current Trade Signals: {'buy_times': ['23:06', '23:10', '23:12', '23:14', '23:16', '23:18', '23:20', '23:24', '23:29'], 'sell_times': ['23:05', '23:08', '23:11', '23:13', '23:15', '23:17', '23:19', '23:23', '23:26', '23:31'], 'half_sell_times': ['23:05', '23:09', '23:11', '23:13', '23:15', '23:17', '23:19', '23:23', '23:27']}
📌 Current Time (Chicago Time): 2025-03-10 23:05:00-05:00
📌 Current Market Price: 67.01
📌 Checking Buy Time: 2025-03-10 23:06:00-05:00 (Expected Execution Time)
⏳ No match for 2025-03-10 23:06:00-05:00. Current Chicago Time: 2025-03-10 23:05:00-05:00
📌 Checking Buy Time: 2025-03-10 23:10:00-05:00 (Expected Execution Time)
⏳ No match for 2025-03-10 23:10:00-05:00. Current Chicago Time: 2025-03-10 23:05:00-05:00
📌 Checking Buy Time: 2025-03-10 23:12:00-05:00 (Expected Execution Time)
⏳ No match for 2025-03-10 23:12:00-05:00. Current Chicago Time: 2025-03-10 23:05:00-05:00
📌 Checking Buy Time: 2025-03-10 23:14:00-05:00 (Expected Execu

KeyboardInterrupt: 

In [31]:

# Define trading parameters
STOP_LOSS_THRESHOLD = 0.99  # Sell if price falls 1% below entry
TRAILING_STOP_THRESHOLD = 0.995  # Sell if price drops 0.5% from peak
TRADE_RISK_PERCENTAGE = 0.2  # Risk 20% of available cash per trade
TRADING_TIMEZONE = ZoneInfo("America/New_York")  # Set timezone to New York

def execute_stockA_trading():
    global trade_times_dict, symbol
    
    print("\n📌 DEBUG: Trading Execution Start")
    print(f"Current Trade Signals: {trade_times_dict}")
    
    now = datetime.now(ZoneInfo("America/New_York")).replace(second=0, microsecond=0)
    print(f"📌 Current Time (NY Time): {now}")

    if not trade_times_dict or not trade_times_dict["buy_times"] or not trade_times_dict["sell_times"]:
        print("⚠️ No trading signals available. Skipping trade execution.")
        return

    # Get latest real-time market data
    df = get_real_time_market_data(symbol)
    current_price = df["close"].iloc[-1]  # Latest market price
    print(f"📌 Current Market Price: {current_price}")

    buy_time = datetime.strptime(trade_times_dict["buy_times"][0], "%H:%M").replace(
        year=now.year, month=now.month, day=now.day, tzinfo=TRADING_TIMEZONE
    )

    sell_time = datetime.strptime(trade_times_dict["sell_times"][0], "%H:%M").replace(
        year=now.year, month=now.month, day=now.day, tzinfo=TRADING_TIMEZONE
    )

    half_sell_time = datetime.strptime(trade_times_dict["half_sell_times"][0], "%H:%M").replace(
        year=now.year, month=now.month, day=now.day, tzinfo=TRADING_TIMEZONE
    )

    # 🚀 Debugging Buy Order Execution
    print(f"📌 Checking Buy Time: {buy_time} (Expected Execution Time)")
    if now == buy_time:
        print(f"✅ Match Found! Executing Buy Order at {now}")

        # Calculate order quantity
        buy_amount = portfolio_cash[symbol] * TRADE_RISK_PERCENTAGE  # 20% of available cash
        qty_to_buy = int(buy_amount / current_price)

        print(f"🔍 Portfolio Cash: {portfolio_cash[symbol]:.2f}, Buy Amount: {buy_amount:.2f}")
        print(f"🔍 Quantity to Buy: {qty_to_buy}, Required Cash: {qty_to_buy * current_price:.2f}")

        if qty_to_buy > 0:
            place_order(symbol, qty=qty_to_buy, side="buy")
            holdings[symbol] += qty_to_buy
            active_trades[symbol] = {
                "buy_price": current_price,
                "highest_price": current_price,
                "trade_time": now
            }
            print(f"✅ {symbol}: Bought {qty_to_buy} shares at {current_price:.2f}")

    # 🚀 Debugging Sell Order Execution
    print(f"📌 Checking Sell Time: {sell_time} (Expected Execution Time)")
    if now == sell_time and holdings[symbol] > 0:
        print(f"✅ Match Found! Executing Sell Order at {now}")
        place_order(symbol, qty=holdings[symbol], side="sell")
        portfolio_cash[symbol] += holdings[symbol] * current_price
        holdings[symbol] = 0  # Exit position
        print(f"✅ {symbol}: Sold ALL at {current_price:.2f}")

    # 🚀 Debugging Half-Sell Order Execution
    print(f"📌 Checking Half-Sell Time: {half_sell_time} (Expected Execution Time)")
    if now == half_sell_time and holdings[symbol] > 0:
        qty_to_sell = int(holdings[symbol] * 0.3)  # Sell 30%
        if qty_to_sell > 0:
            place_order(symbol, qty=qty_to_sell, side="sell")
            holdings[symbol] -= qty_to_sell
            print(f"✅ {symbol}: Sold 30% ({qty_to_sell} shares) at {current_price:.2f}")

    # **Risk Management (Stop-Loss & Trailing Stop)**
    if symbol in active_trades:
        trade_data = active_trades[symbol]
        highest_price = max(trade_data["highest_price"], current_price)

        # Update highest price seen
        active_trades[symbol]["highest_price"] = highest_price

        # Stop-Loss Check
        if current_price <= trade_data["buy_price"] * STOP_LOSS_THRESHOLD:
            print(f"⚠️ Stop-loss triggered for {symbol}. Selling all holdings.")
            place_order(symbol, qty=holdings[symbol], side="sell")
            holdings[symbol] = 0
            print(f"⚠️ {symbol}: Stop-loss triggered. Sold at {current_price:.2f}")

        # Trailing Stop Check
        elif current_price <= highest_price * TRAILING_STOP_THRESHOLD:
            print(f"⚠️ Trailing stop triggered for {symbol}. Selling all holdings.")
            place_order(symbol, qty=holdings[symbol], side="sell")
            holdings[symbol] = 0
            print(f"⚠️ {symbol}: Trailing stop triggered. Sold at {current_price:.2f}")


In [37]:
start_time = time.time()
while time.time() - start_time < 600:
    execute_stockA_trading()
    time.sleep(60)


📌 DEBUG: Trading Execution Start
Current Trade Signals: {'buy_times': ['23:42'], 'sell_times': ['00:05'], 'half_sell_times': ['23:54']}
📌 Current Time (NY Time): 2025-03-11 00:43:00-04:00
📌 Current Market Price: 67.01
📌 Checking Buy Time: 2025-03-11 23:42:00-04:00 (Expected Execution Time)
📌 Checking Sell Time: 2025-03-11 00:05:00-04:00 (Expected Execution Time)
📌 Checking Half-Sell Time: 2025-03-11 23:54:00-04:00 (Expected Execution Time)

📌 DEBUG: Trading Execution Start
Current Trade Signals: {'buy_times': ['23:42'], 'sell_times': ['00:05'], 'half_sell_times': ['23:54']}
📌 Current Time (NY Time): 2025-03-11 00:44:00-04:00
📌 Current Market Price: 67.01
📌 Checking Buy Time: 2025-03-11 23:42:00-04:00 (Expected Execution Time)
📌 Checking Sell Time: 2025-03-11 00:05:00-04:00 (Expected Execution Time)
📌 Checking Half-Sell Time: 2025-03-11 23:54:00-04:00 (Expected Execution Time)

📌 DEBUG: Trading Execution Start
Current Trade Signals: {'buy_times': ['23:42'], 'sell_times': ['00:05'], 'ha

In [35]:
active_trades ={}